In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

In [ ]:
input_name = r'\Inputs_TEST.xlsx'
hyp_name = '\Model_hypotheses.xlsx'
data = r'\221122_TUV_test_conditions_S425-I_MPP.xlsx'

output_name = 'TEST_n1'

par = pr.import_input(r'G:\Drive partagés\Cercle Hard\Innovation\PVT-perf-1Dmodel\Inputs',input_name)
hyp = pr.create_dict_from_excel(r'G:\Drive partagés\Cercle Hard\Innovation\PVT-perf-1Dmodel\Hypotheses',hyp_name,'Main')
condi = pr.create_inputs_from_excel(r'G:\Drive partagés\Cercle Hard\Innovation\PVT-perf-1Dmodel\Steady-state conditions inputs'+data,par,hyp)

In [ ]:
par_p = ty.write_par_p_from_condi(condi,0,hyp)

In [ ]:
par_p

In [ ]:
# slices_df,df_one,its_df_list = ty.simu_one_steady_state(par,par_p,hyp)

In [ ]:
df_one,res = ty.simu_one_steady_state_all_he(par,par_p,hyp)

In [ ]:
bal,bal_perc = ty.recap_energy_balances(res['inlet_man']['df_one'])
print('inlet',bal)
bal,bal_perc = ty.recap_energy_balances(res['exchanger']['df_one'])
print('exchanger',bal)
bal,bal_perc = ty.recap_energy_balances(res['outlet_man']['df_one'])
print('outlet',bal)

In [ ]:
pr.disp_html(res['inlet_man']['df_one'])

In [ ]:
pfun.plot_graph(res['inlet_man']['df_one'])

In [ ]:
pfun.plot_graph(res['exchanger']['df_one'])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_diagram(W, Htube, Dtube, L, w_tube, l_plate, upper_gap, abs_gap, insulation_thickness):
    fig, ax = plt.subplots()

    # Draw the main rectangle (the outer box)
    main_rectangle = patches.Rectangle((0, 0), W, Htube, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(main_rectangle)

    # Draw the tube
    tube = patches.Rectangle((W/2 - w_tube/2, upper_gap + abs_gap), w_tube, Dtube, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(tube)
    
    # Draw the riser plate
    riser_plate = patches.Rectangle((W/2 - w_tube/2, upper_gap + abs_gap + Dtube), w_tube, l_plate, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(riser_plate)
    
    # Add insulation below
    insulation = patches.Rectangle((0, Htube - insulation_thickness), W, insulation_thickness, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(insulation)

    # Set the limits of the plot
    ax.set_xlim(-10, W + 10)
    ax.set_ylim(-10, Htube + 10)

    # Set equal scaling
    ax.set_aspect('equal')

    # Remove the axes
    ax.axis('off')

    # Add dimension lines (using arrows)
    ax.annotate("", xy=(0, Htube + 5), xytext=(W, Htube + 5), arrowprops=dict(arrowstyle='<->'))
    ax.text(W/2, Htube + 6, f"W={W}", ha='center')

    ax.annotate("", xy=(-5, 0), xytext=(-5, Htube), arrowprops=dict(arrowstyle='<->'))
    ax.text(-6, Htube/2, f"Htube={Htube}", va='center', rotation='vertical')

    ax.annotate("", xy=(W/2 + w_tube/2 + 1, upper_gap + abs_gap), xytext=(W/2 + w_tube/2 + 1, upper_gap + abs_gap + Dtube), arrowprops=dict(arrowstyle='<->'))
    ax.text(W/2 + w_tube/2 + 2, upper_gap + abs_gap + Dtube/2, f"Dtube={Dtube}", va='center', rotation='vertical')

    # Save the figure
    plt.show()


draw_diagram(W=300, Htube=100, Dtube=30, L=150, w_tube=50, l_plate=20, upper_gap=10, abs_gap=5, insulation_thickness=10)


In [ ]:
pr.disp_html(df_one)

In [ ]:
df_one['T_PV_absfin_mean']

In [ ]:
df_one['Qdot_PV_absfin']

In [ ]:
df_one['Qdot_absfin_Base']

In [ ]:
df_one['Qdot_absfin_back_conv']

In [ ]:
df_one['Qdot_absfin_back_rad']

In [ ]:
df_one['h_back']

In [ ]:
pr.disp_html(its_df_list[0])

In [ ]:
slices_df['T_Base_mean']

In [ ]:
slices_df['T_B_check']

In [ ]:
bal

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes
nodes = ['T_amb', 'T_sky', 'T_glass', 'T_PV', 'T_B', 'T_absfin', 
         'T_tube', 'T_fluid', 'T_ins_absfin', 'T_ins_tube', 'T_back', 'T_back_rad']
G.add_nodes_from(nodes)

# Add edges with labels
edges = [('T_amb', 'T_glass', 'Qdot_top_conv'), ('T_sky', 'T_glass', 'Qdot_top_rad'),
         ('T_glass', 'T_PV', 'Qdot_glass_PV'), ('T_PV', 'T_B', 'Qdot_PV_Base'),
         ('T_PV', 'T_absfin', 'Qdot_PV_absfin'), ('T_absfin', 'T_B', 'Qdot_absfin_Base'),
         ('T_B', 'T_tube', 'Qdot_Base_tube'), ('T_tube', 'T_fluid', 'Qdot_tube_fluid'),
         ('T_absfin', 'T_ins_absfin', 'Qdot_absfin_ins_absfin'),
         ('T_tube', 'T_ins_tube', 'Qdot_tube_ins_tube'),
         ('T_ins_absfin', 'T_back', 'Qdot_ins_absfin_back'),
         ('T_ins_absfin', 'T_back_rad', 'Qdot_ins_absfin_back_rad'),
         ('T_ins_tube', 'T_back', 'Qdot_ins_tube_back'),
         ('T_ins_tube', 'T_back_rad', 'Qdot_ins_tube_back_rad')]

for u, v, label in edges:
    G.add_edge(u, v, label=label)

# Manually set positions based on TikZ coordinates
pos = {
    'T_amb': (3, 0),
    'T_sky': (8, 0),
    'T_glass': (5.5, -3),
    'T_PV': (5.5, -8),
    'T_B': (3, -11),
    'T_absfin': (8, -11),
    'T_tube': (4, -15),
    'T_fluid': (4, -20),
    'T_ins_absfin': (9, -19),
    'T_ins_tube': (6, -19),
    'T_back': (6, -26),
    'T_back_rad': (9, -26)
}

# Draw the graph
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='black')

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.show()


In [ ]:
df_one.columns

In [ ]:
df_one['T_back'].iloc[0]

In [ ]:
df_one['Qdot_sun_PV'] + df_one['Qdot_sun_glass']

In [ ]:
- df_one['Qdot_top_conv'] - df_one['Qdot_top_rad'] - df_one['Qdot_PV_sky'] - df_one['Qdot_absfin_back_conv'] - df_one['Qdot_absfin_back_rad'] - df_one['Qdot_Base_back'] - df_one['Qdot_tube_back_conv'] - df_one['Qdot_tube_back_rad'] - df_one['Qdot_tube_sky']

In [ ]:
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_Base_back', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']

In [ ]:
ht_signs = {'Qdot_sun_PV':1, 'Qdot_sun_glass':1, 'Qdot_top_conv':-1, 'Qdot_top_rad':-1, 'Qdot_PV_sky':-1, 'Qdot_absfin_back_conv':-1, 'Qdot_absfin_back_rad':-1, 'Qdot_Base_back':-1, 'Qdot_tube_back_conv':-1, 'Qdot_tube_back_rad':-1, 'Qdot_tube_sky':-1}

In [ ]:
df_one['Qdot_tube_fluid']

In [ ]:
data = {key:ht_signs[key]*df_one[key].values[0] for key in ht_labels}

In [ ]:
# Adjusted approach for a single-column histogram with offset bars
fig = go.Figure()

# Initialize cumulative sums for positive and negative values
cumulative_positive = 0
cumulative_negative = 0

for key, value in data.items():
    if value >= 0:
        # For positive values, start from the last positive cumulative sum
        base = cumulative_positive
        cumulative_positive += value
    else:
        # For negative values, start from the last negative cumulative sum
        base = cumulative_negative
        cumulative_negative += value
    
    fig.add_trace(go.Bar(
        x=["Heat Transfer Contributions"],
        y=[value],
        name=key,
        width=0.2,
        base=base  # Set the starting point of the bar
    ))

# Update the layout
fig.update_layout(
    title="Decomposition of Qdot_tube_fluid in Different Heat Transfer Contributions",
    yaxis_title="Value",
    barmode="overlay"  # Overlay the bars
)

# Show the figure
fig.show()


In [ ]:
df_one['Q_PV_absfin']

In [ ]:
df_one['Q_absfin_Base']

In [ ]:
df_one['h_back']

In [ ]:
df_one['Q_absfin_back_rad'] 

In [ ]:
df_one['Q_absfin_back_conv']

In [ ]:
df_one['h_rad_back']

In [ ]:
df_one['T_absfin_mean']

In [ ]:
df_one['Q_Base_tube']

In [ ]:
df_one['Q_tube_fluid']

In [ ]:
df_one['Q_tube_back_conv']

In [ ]:
df_one['Q_tube_back_rad']

In [ ]:
df_one['Q_tube_sky']

In [ ]:
slices_df['T_tube_mean']

In [ ]:
slices_df['T_glass']

In [ ]:
slices_df['T_amb']

In [ ]:
df_one['Q_PV_Base']

In [ ]:
df_one['Q_absfin_Base'] 

In [ ]:
df_one['Q_Base_tube']

In [ ]:
df_one['Q_Base_back']

In [ ]:
df_one['Q_PV_Base'] + df_one['Q_Base_tube']

In [ ]:
df_one['T_PV']

In [ ]:
df_one['T_Base_mean']

In [ ]:
df_one['T_tube_mean']

In [ ]:
df_one['T_fluid_mean']

In [ ]:
6.09/9.84

In [ ]:
df_one['Q_tube_sky']

In [ ]:
df_one['Q_Base_tube']

In [ ]:
df_one['Q_tube_fluid']

In [ ]:
df_one['Q_tube_back_conv']

In [ ]:
df_one['Q_tube_back_rad']

In [ ]:
df_one['Q_glass_PV']

In [ ]:
df_one['Q_sun_glass']

In [ ]:
df_one['Q_top_conv']

In [ ]:
df_one['Q_top_rad']

In [ ]:
df_one['Q_PV_Base']

In [ ]:
df_one['Q_PV_absfin']

In [ ]:
df_one['Q_PV_plate']